In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import os
import re
import flywheel

In [4]:
fw = flywheel.Client()

In [12]:
project = fw.lookup('scien/tome')

In [17]:
session = project.sessions()[0]

In [8]:
def _dcm_from_archive_slow(file_entry, acq):
    """
    For a given file entry, read a dicom file from the archive and return a pydicom object.
    Method here will read the dicom file from the archive as raw data and read that raw data into pydicom
    """
    import pydicom
    from pydicom.filebase import DicomBytesIO
   
    # Check that this file is valid
    if file_entry['type'] != 'dicom':
        raise TypeError('must be a dicom file!')
   
    # This reads the raw dicom data stream into a pydicom object
    #     (https://github.com/pydicom/pydicom/issues/653#issuecomment-449648844)
    zip_info = acq.get_file_zip_info(file_entry['name'])
    acq.download_file_zip_member(file_entry['name'], zip_info.members[1].path, '/tmp/dcm0.dcm')
    dcm = pydicom.dcmread('/tmp/dcm0.dcm')
   
    return dcm

In [9]:
def _dcm_from_archive(file_entry, acq):
    """
    For a given file entry, read a dicom file from the archive and return a pydicom object.
    Method here will read the dicom file from the archive as raw data and read that raw data into pydicom
    """
    import pydicom
    from pydicom.filebase import DicomBytesIO
   
    # Check that this file is valid
    if file_entry['type'] != 'dicom':
        raise TypeError('must be a dicom file!')
   
    # This reads the raw dicom data stream into a pydicom object
    #     (https://github.com/pydicom/pydicom/issues/653#issuecomment-449648844)
    zip_info = acq.get_file_zip_info(file_entry['name'])
    raw_dcm = DicomBytesIO(acq.read_file_zip_member(file_entry['name'], zip_info.members[1].path))
    dcm = pydicom.dcmread(raw_dcm)
   
    return dcm

## Test on a small archive

In [20]:
acq = session.acquisitions.find('label=Localizer')[0]

In [21]:
file_entry = {'name': 'Localizer.dcm.zip', 'type': 'dicom'}

In [22]:
%%timeit
_dcm_from_archive(file_entry, acq)

1.48 s ± 202 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
_dcm_from_archive_slow(file_entry, acq)

1.4 s ± 168 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Test on a large archive

In [26]:
acq = session.acquisitions.find('label=rfMRI_REST_PA_Run4')[0]

In [27]:
file_entry = {'name': 'rfMRI_REST_PA.dcm.zip', 'type': 'dicom'}

In [28]:
%%timeit
_dcm_from_archive(file_entry, acq)

4 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
_dcm_from_archive_slow(file_entry, acq)

3.97 s ± 296 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
